# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dudinka,69.4058,86.1778,18.78,58,98,4.75,RU,1722212028
1,1,amga,60.8953,131.9608,19.89,59,99,2.83,RU,1722212029
2,2,thompson,55.7435,-97.8558,21.21,53,91,1.97,CA,1722212030
3,3,grytviken,-54.2811,-36.5092,-2.35,69,100,7.22,GS,1722212031
4,4,luderitz,-26.6481,15.1594,11.36,64,0,7.93,NaN,1722212033


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [45]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 810,
    frame_height = 605,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=12)\
                            &(city_data_df['Max Temp']>=0)\
                            &(city_data_df['Cloudiness']<=90)\
                            &(city_data_df['Country']!='US')\
                            &(city_data_df['Country']!='AU')\
                            &(city_data_df['Country']!='RU')\
                            &(city_data_df['Country']!='IN')\
                            &(city_data_df['Country']!='HK')\
                            &(city_data_df['Humidity']<70),:]
print(len(ideal_df))

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head()

8


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,luderitz,-26.6481,15.1594,11.36,64,0,7.93,NaN,1722212033
110,110,chui,-33.6971,-53.4616,7.71,69,54,4.00,UY,1722212176
139,139,san agustin de valle fertil,-30.6335,-67.4682,8.04,48,73,2.41,AR,1722212214
187,187,dolores,-36.3132,-57.6792,4.91,44,0,3.07,AR,1722212276
315,315,comodoro rivadavia,-45.8667,-67.5000,5.92,45,0,4.63,AR,1722212432


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
4,luderitz,NaN,-26.6481,15.1594,64,
110,chui,UY,-33.6971,-53.4616,69,
139,san agustin de valle fertil,AR,-30.6335,-67.4682,48,
187,dolores,AR,-36.3132,-57.6792,44,
315,comodoro rivadavia,AR,-45.8667,-67.5000,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
luderitz - nearest hotel: Seaview Hotel Zum Sperrgebiet
chui - nearest hotel: Alerces
san agustin de valle fertil - nearest hotel: Cabañas Valle Pintado
dolores - nearest hotel: Plaza Hotel
comodoro rivadavia - nearest hotel: No hotel found
queenstown - nearest hotel: Queens Hotel
gualeguay - nearest hotel: No hotel found
sarmiento - nearest hotel: Ismar


,City,Country,Lat,Lng,Humidity,Hotel Name
4,luderitz,NaN,-26.6481,15.1594,64,Seaview Hotel Zum Sperrgebiet
110,chui,UY,-33.6971,-53.4616,69,Alerces
139,san agustin de valle fertil,AR,-30.6335,-67.4682,48,Cabañas Valle Pintado
187,dolores,AR,-36.3132,-57.6792,44,Plaza Hotel
315,comodoro rivadavia,AR,-45.8667,-67.5000,45,No hotel found
409,queenstown,ZA,-31.8976,26.8753,60,Queens Hotel
438,gualeguay,AR,-33.1416,-59.3097,56,No hotel found
490,sarmiento,AR,-45.5882,-69.0700,62,Ismar


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [47]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)